# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [66]:
import uuid
!pip install langchain langchain-groq python-dotenv chromadb langchain-community langchain-core streamlit

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import dotenv
import os

from langchain_groq import ChatGroq

dotenv.load_dotenv()

llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv('GROQ_API_KEY'),
    model="llama-3.1-8b-instant",
)

In [7]:
response = llm.invoke("Tell me something about moon")
print(response.content)

The Moon is Earth's only natural satellite and has been a subject of human fascination for centuries. Here are some interesting facts about the Moon:

1. **Formation**: The Moon is believed to have formed about 4.5 billion years ago, not long after the formation of the Earth. One theory is that a massive object collided with the early Earth, causing debris to be thrown into orbit and eventually coalesce into the Moon.

2. **Size and Distance**: The Moon is relatively small, with a diameter of about 2,159 miles (3,475 kilometers). It is also relatively close to Earth, with an average distance of about 238,855 miles (384,400 kilometers).

3. **Phases**: The Moon goes through eight distinct phases, which are determined by its position relative to the Sun and Earth. These phases include new moon, waxing crescent, first quarter, waxing gibbous, full moon, waning gibbous, last quarter, and waning crescent.

4. **Gravity**: The Moon's gravity is only about one-sixth of Earth's, which is why a

In [36]:
from langchain_community.document_loaders import WebBaseLoader

In [37]:
loader = WebBaseLoader("https://jobs.nike.com/job/R-39383", header_template={
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
  })
page_data = loader.load().pop().page_content


In [38]:
page_data

"Apply for Software Engineer II - Workday Integration, ITC\n\nSearch JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer II - Workday Integration, ITCKarnataka, IndiaBec

In [45]:
from langchain_core.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template(
    f"""
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data} 
    ### INSTRUCTION: 
    The scraped text is from the career's page of a website. Your job is to extract the job postings and return them in JSON format containing following keys: 'role', 'skills', 'experience' and 'description' .
    Only return the valid JSON.
    ### OUTPUT FORMAT:
    JSON format containing following keys: 'role', 'skills', 'experience' ,'description' (NO PREAMBLE)
    """
)

In [46]:
from langchain_core.output_parsers import JsonOutputParser

chain_extract = prompt_template | llm
res = chain_extract.invoke(input={'page_data': page_data})
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

In [47]:
json_res

[{'role': 'Software Engineer II - Workday Integration, ITC',
  'skills': ['Workday Integrations with Workday Studio, EIB, Core Connectors and payroll connectors (PECI) and their related tools such as XML, XSLT',
   'Web Services (WSDL, SOAP), EDI and Flat file integrations',
   'SaaS applications, SaaS deployment models, and SaaS DevOps',
   'Object Oriented concepts',
   'Workday Studio Certification',
   'Agile Framework'],
  'experience': {'required': '5+ years of experience building integrations and software development',
   'specific': {'workday': '3+ Workday Integrations',
    'third-party': "3+ years' experience integrating systems with third-party service vendors"}},
  'description': 'As a Workday Software Integration Engineer, you will work with the team of engineers working on integrations across products. You will be responsible to develop, test, and deploy integrations required to meet business requirements. You will be driving the technical deliverables to be delivered on 

In [27]:
import pandas as pd

df = pd.read_csv('data/job_portfolio.csv')

df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [29]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row['Techstack'], metadatas= {"link": row['Links']}, ids=[str(uuid.uuid4())])

In [32]:
links = collection.query(query_texts=['Experience in Python', 'Experience in Java'], n_results=1).get('metadatas')
links

[[{'link': 'https://example.com/ml-python-portfolio'}],
 [{'link': 'https://example.com/java-portfolio'}]]

In [56]:
job = json_res
job[0]['skills']

['Workday Integrations with Workday Studio, EIB, Core Connectors and payroll connectors (PECI) and their related tools such as XML, XSLT',
 'Web Services (WSDL, SOAP), EDI and Flat file integrations',
 'SaaS applications, SaaS deployment models, and SaaS DevOps',
 'Object Oriented concepts',
 'Workday Studio Certification',
 'Agile Framework']

In [64]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Sumaya, a business development executive at ACME AI. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of ACME AI 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase ACME's portfolio: {link_list}
        Remember you are Sumaya, BDE at ACME AI. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

In [65]:
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job[0]['skills']), "link_list": links})
print(res.content)

Subject: Unlock Seamless Business Process Integration with ACME AI

Dear [Client Name],

I came across your job description for a Workday Integration Specialist, and I'm excited to introduce you to ACME AI, a leading AI & Software Consulting company dedicated to streamlining business processes through automated tools.

At ACME AI, we've empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise spans Workday Integrations with Workday Studio, EIB, Core Connectors, and payroll connectors (PECI), as well as related tools such as XML, XSLT, Web Services (WSDL, SOAP), EDI, and Flat file integrations.

Our team is well-versed in Object Oriented concepts and Agile Framework, ensuring that our solutions are scalable, maintainable, and aligned with your business goals. We're proud to offer Workday Studio Certification, guaranteeing that our integrations meet the highest standards of qualit

In [68]:
!pip freeze > requirements.txt